In [7]:
import mediapipe as mp
import cv2
import csv
import os
import numpy as np 

In [13]:
cam = cv2.VideoCapture(-1)
cam.isOpened() # True

[ WARN:0@961.286] global cap_v4l.cpp:969 open VIDEOIO(V4L2): can't find camera device
[ERROR:0@961.286] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range


False

In [4]:
import mediapipe as mp
import cv2
import csv
import os
import numpy as np 

#******START CAPTURING THE VIDEO , DETECTING THE LANDMARKS AND CONNECTING THEM.*******

mp_drawing = mp.solutions.drawing_utils # drawing skeleton
mp_holistic = mp.solutions.holistic # pose advanced version

cap = cv2.VideoCapture(0) # webcam on

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic: # call holistic method
    
    while cap.isOpened():
        ret, frame = cap.read()

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # BGR -> RGB
        image.flags.writeable = False  # 이미지 다시쓰기 막음. 읽기 전용.
        results = holistic.process(image)      
        image.flags.writeable = True   # 이미지 다시쓰기 허용.     
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # RGB -> BGR
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
                        
        cv2.imshow('이건 왜 찍는거지..?', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

#**********RECORDING THE CAPTURED COORDINATES IN THE CVS FILE.********   
     
num_coords = len(results.pose_landmarks.landmark) #landmark 수
num_coords
landmarks = ['class'] # class 라는 attribute 를 만들고 그 밑으로 x,y,z,v 를 저장
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]
print(landmarks)
with open('/Users/user/Desktop/landmarks_coordinates.csv', mode='w', newline='') as f: 
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

#******USER DEFINED FUNCTION FOR CALLING THE CAPTURE FUNCTION AGAIN AND AGAIN AND RECORDING THE DATA IN THE CSV.*******.

def Capture(title):
  
  cap = cv2.VideoCapture(0)

  with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        results = holistic.process(image)

        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
  
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        try:
            
            pose = results.pose_landmarks.landmark # 랜드마크 기록
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
           
            row = pose_row

            row.insert(0,title)
            
            with open('/Users/user/Desktop/landmarks_coordinates.csv', mode='a', newline='') as f: #파일 이어쓰기
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row) 
            
        except:
            pass
                        
        cv2.imshow('자세측정. 차례대로 가운데, 왼쪽, 오른쪽', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

  cap.release()
  cv2.destroyAllWindows()
  return

#******CAPTURE AS PER YOUR REQUIREMENT AND PROVIDE THE POSING DATA FOR EACH POSE *******.

Capture(title="가운데")
Capture(title="왼쪽")
Capture(title="오른쪽")



[ WARN:0@249.664] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@249.664] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range


In [1]:
import cv2
import numpy as np
import streamlit as st
import mediapipe as mp

st.title("Image Processing")
st.write(
    "Upload an image below and I will annotate the faces in the image using [MediaPipe](https://mediapipe.dev/)."
)

file = st.file_uploader("Upload a file")
if file is None:
    file = st.camera_input("Or take a picture")

if file is not None:
    # Display original image
    st.write("You uploaded", file.name)
    col1, col2 = st.columns(2)
    col1.image(file)

    # Annotate image
    file_bytes = np.asarray(bytearray(file.read()), dtype=np.uint8)
    image = cv2.imdecode(file_bytes, 1)

    with mp.solutions.face_mesh.FaceMesh(static_image_mode=True) as face_mesh:
        results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # Display annotate image
        if results.multi_face_landmarks:
            annotated_image = image.copy()

            for face_landmarks in results.multi_face_landmarks:
                # st.code(face_landmarks)

                mp.solutions.drawing_utils.draw_landmarks(
                    image=annotated_image,
                    landmark_list=face_landmarks,
                    connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp.solutions.drawing_styles.get_default_face_mesh_tesselation_style(),
                )
                mp.solutions.drawing_utils.draw_landmarks(
                    image=annotated_image,
                    landmark_list=face_landmarks,
                    connections=mp.solutions.face_mesh.FACEMESH_CONTOURS,
                    landmark_drawing_spec=None,
                    connection_drawing_spec=mp.solutions.drawing_styles.get_default_face_mesh_contours_style(),
                )

            col2.image(annotated_image, channels="BGR")
        else:
            col2.write("No faces found")

ModuleNotFoundError: No module named 'streamlit'